In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import datetime as dt
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest

# Assignment 1

In [19]:
df = pd.read_csv('data/meal_conj.csv')

### 1.1

In [20]:
# check missing values
df.isna().sum()

resID         0
main          0
side_dish     0
packaging     0
rating       10
dtype: int64

In [21]:
# drop missing values 
df = df.dropna()

In [22]:
# check duplicates
df.duplicated().sum()

73

In [23]:
#drop duplicated values
df = df.drop_duplicates()

In [24]:
# rating should be positive, check
df[df['rating']<0]
# no negative values

,resID,main,side_dish,packaging,rating


## 1.2

In [25]:
# select dependent and independent variables
y = df['rating']
x = df[['main', 'side_dish', 'packaging']]
# make categort
x = x.astype('str')
# make categories a dummy variables
xdum = pd.get_dummies(x)

In [26]:
# add the constant for a reference point
xdum = sm.add_constant(xdum)
# fit linear regression
res = sm.OLS(y,xdum).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     10.08
Date:                Mon, 06 Mar 2023   Prob (F-statistic):           2.64e-18
Time:                        17:07:43   Log-Likelihood:                -15.130
No. Observations:                3383   AIC:                             54.26
Df Residuals:                    3371   BIC:                             127.8
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.2766      0.002    120.719      0.000       0.272       0.281
main_1         -0.0361      0.009     -3.818      0.000      -0.055      -0.018
main_2          0.0325      0.010      3.340      0.001       0.013       0.052
main_3          0.0538      0.010      5.560      0.000       0.035       0.073
main_4          0.0621      0.009      6.733      0.000       0.044       0.080
main_5          0.0667      0.009      7.165      0.000       0.048       0.085
main_6          0.0976      0.009     10.871      0.000       0.080       0.115
side_dish_1     0.0525      0.009      5.658      0.000       0.034       0.071
side_dish_2     0.0451      0.010      4.642      0.000       0.026       0.064
side_dish_3     0.0637      0.009      6.997      0.000       0.046       0.082
side_dish_4     0.0443      0.009      4.686      0.000       0.026       0.063
side_dish_5     0.0516      0.009      5.548      0.000       0.033       0.070
side_dish_6     0.0193      0.009      2.045      0.041       0.001       0.038
packaging_1     0.1350      0.004     31.217      0.000       0.127       0.144
packaging_2     0.1416      0.004     32.481      0.000       0.133       0.150
==============================================================================
Omnibus:                      100.641   Durbin-Watson:                   1.824
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               57.508
Skew:                          -0.156   Prob(JB):                     3.25e-13
Kurtosis:                       2.442   Cond. No.                     3.01e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.83e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## 1.3

In [27]:
# create a new dataframe with attribute/level pairs and their coefficient scores, which are the same as utility gains
df_res = pd.DataFrame({'param_name':res.params.keys(), 'param_w':res.params.values, 
                       'pval':res.pvalues}).reset_index(drop=True)

In [28]:
# separate attributes and levels
df_res['attr'] = df_res['param_name'].apply(lambda x: x.split('_')[0])
df_res['level'] = df_res['param_name'].apply(lambda x: x.split('_')[-1])

In [29]:
# go over unique attributes and calculate their utility range
attr_utils = {}
for i in list(df_res['attr'].unique())[1:]:
    a = df_res[df_res['attr']==i]
    attr_utils[i]=a['param_w'].max()-a['param_w'].min()

In [30]:
# transform the dictionary into the dataframe of attribute ranges
attr_utils_df = pd.DataFrame(attr_utils, index=[0]).T.rename(columns={0:'utils'})
# calculate the relative importance
attr_utils_df['rel_imp'] = attr_utils_df['utils'].apply(lambda x: (x/attr_utils_df['utils'].sum())*100)

In [31]:
attr_utils_df

,utils,rel_imp
main,0.133628,72.384170
side,0.044439,24.071829
packaging,0.006543,3.544001


In [32]:
fig = px.bar(attr_utils_df, x='utils', title='Utility Range')
fig.update_yaxes(title='Attribute')
fig.show()

In [33]:
fig = px.bar(attr_utils_df, x='rel_imp', title='Relative Importance')
fig.update_yaxes(title='Attribute')
fig.show()

## 1.4

In [34]:
fig = px.bar(df_res.iloc[1:], x="attr", y="param_w", color="level",  barmode='group',)
fig.show()

The ideal product (bundle) will be composed of main=6, side=3, and packaging=2

## 1.5

In [35]:
# select the best combinations
best_bundle=[1,0,0,0,0,0,1,0,0,1,0,0,0,0,1]
pred_rat=res.predict(best_bundle)
print('The predicted rating of the best bundle:', round(pred_rat[0],3))

The predicted rating of the best bundle: 0.58


# Assignment 2

In [36]:
df = pd.read_csv('data/ab_users.csv')

## 2.1

In [37]:
df.isna().sum()

Users       0
Response    0
Group       0
dtype: int64

In [38]:
df.duplicated().sum()

0

## 2.2

#### Null hypothesis: The change of web page didn't have a significant impact on user responses. 
#### Alt. Hypothesis: The change of web page had a significant impact on user responses.

## 2.3

In [39]:
df['Group'].value_counts()

control      500
treatment    468
Name: Group, dtype: int64

In [40]:
ab_control = df[df['Group']=='control']
ab_treatment = df[df['Group']=='treatment']

In [41]:
# randomly split the same proportion from control group equal to the treatment one
ab_control= ab_control.sample(len(ab_treatment),random_state=42)

In [42]:
# create list of successes for each group, how many people converted out of the whole group
successes = [ab_control['Response'].sum(), ab_treatment['Response'].sum()]
# create count of observations
nobs = [len(ab_control), len(ab_treatment)]

In [43]:
zstat, pvalue = proportions_ztest(successes, nobs)
print('zstat: ', zstat)
print('pvalue: ', pvalue)

zstat:  -0.49761335152811853
pvalue:  0.6187565913004276


Since our p-value=0.62 is higher than our α=0.05 threshold, we cannot reject the Null hypothesis Hₒ, which means that our new web design did not perform significantly different than our old one.

# Assignment 3

In [44]:
orders = pd.read_csv('data/List of Orders.csv')
order_details = pd.read_csv('data/Order Details.csv')

## 3.1

In [45]:
print("Missing values in orders dataset:")
print(orders.isnull().sum())
print("\nMissing values in order details dataset:")
print(order_details.isnull().sum())

print("\nNumber of duplicates in orders dataset:", orders.duplicated().sum())
print("Number of duplicates in order details dataset:", order_details.duplicated().sum())


Missing values in orders dataset:
Order ID        60
Order Date      60
CustomerName    60
State           60
City            60
dtype: int64

Missing values in order details dataset:
Order ID        0
Amount          0
Profit          0
Quantity        0
Category        0
Sub-Category    0
dtype: int64

Number of duplicates in orders dataset: 59
Number of duplicates in order details dataset: 0


In [46]:
# remove missing values and duplicates
orders = orders.dropna()
orders = orders.drop_duplicates()

In [47]:
# check for missing values in amount and quantity columns
print('Amount negative values:',len(order_details[order_details['Amount']<0]))
print('Quantity negative values:',len(order_details[order_details['Quantity']<0]))

Amount negative values: 0
Quantity negative values: 0


## 3.2 

In [48]:
# merge data
orders_all=pd.merge(orders, order_details, on='Order ID')

In [49]:
orders_all.head()

,Order ID,Order Date,CustomerName,State,City,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,66.0,-12.0,5,Clothing,Stole
2,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,01-04-2018,Pearl,Maharashtra,Pune,168.0,-111.0,2,Electronics,Phones


In [50]:
# specify the datetime format 
orders_all['Order Date'] =pd.to_datetime(orders_all['Order Date'])
# there is lack of information about amount, profit and quantity, 
#so I assume that amount is the total spending for that order

/Users/arsinesarikyan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning:

Parsing '13-04-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/Users/arsinesarikyan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning:

Parsing '15-04-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/Users/arsinesarikyan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning:

Parsing '17-04-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/Users/arsinesarikyan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning:

Parsing '18-04-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/Users/arsinesarikyan/opt/anaconda3/lib/python3.8/site-p

In [51]:
# see if the same person can order from different locations
orders_all.groupby(by=['CustomerName'])['State'].nunique().sort_values(ascending=False)

CustomerName
Shreya      5
Abhishek    5
Shubham     5
Rohan       4
Pooja       4
           ..
Jaydeep     1
Jayanti     1
Jay         1
Jaideep     1
Yohann      1
Name: State, Length: 332, dtype: int64

It is evident that people with the same name can be in different states, so in order to have a better ID, we can create a new column based on name and location

In [52]:
orders_all['id'] = orders_all['CustomerName']+"_"+orders_all['State']

## 3.3

In [55]:
# create rfm values based on general practices
rfm = orders_all.groupby(by='id').agg({'Order Date': lambda date:(orders_all['Order Date'].max()- date.max()).days,
                                 'Order ID': 'count', 'Amount':'sum'})
rfm.columns = ['recency', 'frequency', 'monetaryvalue']

## 3.4

In [56]:
rfm['frequency'].value_counts()

1     151
3      48
4      40
5      35
2      29
6      23
8      17
7      16
9      15
10     10
12      4
13      4
11      3
14      3
16      1
Name: frequency, dtype: int64

In [160]:
# create quantile values such as 1 will be the best option
rfm['r_q'] = pd.qcut(rfm['recency'],4,['1','2','3','4']) # recent purchases are good
rfm['r_f'] = pd.qcut(rfm['frequency'],2,['2','1'])# high number of purchases
rfm['r_m'] = pd.qcut(rfm['monetaryvalue'],4,['4','3','2','1']) # high number of monetary values

the distribution of frequency is very narrow, that's why it was impossible to divide into higher groups, I decided that for this task it can be useful to separate them just into 2 groups

In [161]:
# join the quantiles and create the score 
rfm['RFM_score'] = rfm['r_q'].astype(str)+rfm['r_f'].astype(str)+rfm['r_m'].astype(str)

## 3.5

In [179]:
rfm[rfm['RFM_score']=='424'].head()

,recency,frequency,monetaryvalue,r_q,r_f,r_m,RFM_score
id,,,,,,,
Aakanksha_Madhya Pradesh,695,2,74.0,4,2,4,424
Aayushi_Madhya Pradesh,606,1,73.0,4,2,4,424
Aditi_Rajasthan,600,1,139.0,4,2,4,424
Akanksha_Andhra Pradesh,554,1,27.0,4,2,4,424
Akash_West Bengal,495,1,131.0,4,2,4,424


- ***'111'*** = Top/Best customers- loyal, frequent and oftern buyers who bring high value and are crucial for the business.
- ***'311'*** = High Value Customer- loyal but not so frequent buyets, they may not purchase recently buy overall they generate a lot of monetary value. Strategies can be designed to decrease the time between their purchases
- ***'424'*** = Lost Customer- these customers made a purchase a while ago, they are not frequent buyers and spent little money. Even though understanding them is crucial it is not the top priority.